# Mine Production Planning with Stockpiling (with Pyomo)

## Introduction
Open-pit mine production scheduling is a central planning task in mining operations. The goal is to decide **when and how much material to extract and process** from different blocks of the mine, over a finite time horizon, in order to maximize the economic value of the project. Traditionally, this is formulated as the **Open Pit Mine Production Scheduling Problem (OPMPSP)**, which already is a large-scale mixed-integer optimization problem due to precedence constraints among blocks, mining and processing capacity limits, and economic discounting.

In practice, mining operations often make use of **stockpiles**: mined ore that is not immediately processed can be stored and reclaimed later. This flexibility allows the mine to adapt to fluctuations in ore quality and market conditions, increasing overall Net Present Value (NPV). However, the introduction of stockpiles complicates the optimization model: once material is placed into a stockpile, it is mixed homogeneously with existing material, and when it is reclaimed, the quality corresponds to the **average composition** of the stockpile. Mathematically, this leads to **bilinear (quadratic) constraints**, which are computationally challenging for standard MILP solvers.

The paper by [Bley, Boland, Froyland, and Zuckerberg (2012)](https://web.maths.unsw.edu.au/~froyland/bbfz.pdf) develops advanced formulations (Natural, Aggregated, and Discretized models) to handle stockpile effects within an optimization framework. Their results show that carefully designed extended formulations and branching strategies yield much tighter relaxations and practical solvability compared to naive models.


## Problem Description
We consider the **Open Pit Mine Production Scheduling Problem with Stockpiling (OPMPSP+S).**
The complete problem formulation and context are provided in the attached paper. 
Here, we focus on the implementation and include only brief explanatory comments.

In the paper, the model is presented in a general form without a concrete numerical example. In this notebook, we apply the model to the following practical case.

### Example

We consider a small mine with **5 blocks** to be scheduled over **3 time periods**. Each block is characterized by its rock tonnage, ore tonnage, and metal content, together with precedence constraints that specify the mining order. In each time period, mining and processing capacities limit how much material can be extracted and treated. In addition, a stockpile is available, allowing material to be stored for later processing, but subject to the homogeneous-mixing quality constraint.  

This small dataset allows us to illustrate the full model formulation, decision variables, and constraints in action, and to show how the optimization can be solved using `Pyomo`.

In [157]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import re
from optutils import pyomo_helpers, plotting
from optutils.utils import read_txt, parse_arcs_head_format
from optutils.pyomo_helpers import build_model, add_variables, add_constraints, add_objective
import pyomo.environ as pyo
from pyomo.environ import SolverFactory
from pyomo.environ import value

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load and Process Data

In [2]:
# read parameters_marvin.txt and extract its informations
parameters = read_txt("../data/mine_production_planning/parameters_marvin.txt")

# q
q = parameters["annual_discount"]

# mining capacity and processing capacity
mining_capacity, processing_capacity = parameters["capacity_mining"], parameters["capacity_processing"]

# time_periods
periods = parameters["time_periods"]
periods = list(range(1, periods+1))

# -- Time periods with mining and processing capacities --
# time_data: {t: (mining_cap, processing_cap)}
time_data = {t: (mining_capacity, processing_capacity) for t in periods}

# scalar costs / global (time-invariant)
m = parameters["cost_mining"]                 # mining cost per tonne of rock
proc_cost = parameters["cost_processing"]     # processing cost per tonne of ore
c1 = parameters["price_metal"]                # sales price per tonne of metal
c2 = parameters["price_metal2"]               # sales price2 per tonne of metal

In [3]:
# import blocks with [rock_tonnage, ore_tonnage, metal_tonnage], i.e. [R, O, A]
df = pd.read_csv("../data/mine_production_planning/panels.csv", sep=";")
df.columns = df.columns.str.strip()                  # to avoid whitespace

# drop summary rows (min, max, ang, std-dev)
df = df[pd.to_numeric(df["block"], errors="coerce").notna()].copy()

# ensure numeric types of the columns
df[["block", "rock", "ore", "metal"]] = df[["block", "rock", "ore", "metal"]].apply(pd.to_numeric, errors="coerce")
df["block"] = df["block"].astype(int)

# remove any row where al least one of the columns "rock", "ore", or "metal" is missing (NaN).
df.dropna(subset=["rock", "ore", "metal"], how="any", inplace=True)

# -- Blocks with (rock_tonnage R_i, ore_tonnage O_i, metal_tonnage A_i) --
# A_i corresponds to the contained metal if the entire block is processed.
blocks_data = {
    int(b): (float(r), float(o), float(a)) 
    for b, r, o, a in zip(df["block"], df["rock"], df["ore"], df["metal"])
}

In [4]:
# Define plants for processing to extract the metal from the rock
plants = ["P"]

# Define stockpiles 
stockpiles = ["S"]

# Define waste dumps
waste_dumps = ["W"]

In [5]:
# import precedence constraints from arcs.txt (which i must be mined before j)
# Precedence constraints: i can only advance after all j in P[i] are complete
with open("../data/mine_production_planning/arcs.txt", encoding='utf-8') as f:
    arcs = [line.strip() for line in f]
P = parse_arcs_head_format(arcs, blocks_data)

## I- Aggregate Tracking

### (1) Aggregate Tracking Model (AT)

$$
\begin{aligned}
S_{\mathrm{AT}}
:= \Big\{ & (x, y, z^{P}, z^{S}, z^{S,P}, z^{S,S}, o^{S}, a^{S}, o^{P}, a^{P}, f)
          \in [0,1]^{6(N\times T)} \times \mathbb{R}_{\ge 0}^{4T} \times [0,1]^T : \\
& (x, y, z^{P}, z^{S}, z^{S,P}, z^{S,S}, o^{S}, a^{S}, o^{P}, a^{P}, f)
   \text{ satisfies } (1)\text{--}(10),\ (12)\text{--}(18), \\
& x \in \{0,1\}^{N\times T} \Big\}.
\end{aligned}
$$

In [176]:
M_AT = build_model(time_data, blocks_data, plants, stockpiles, waste_dumps, P, parameters)

# Define decision variables  with x is set to binary
add_variables(M_AT, aggregate=True, x_binary=True)

# Define constraints with enforce_mixing=False
add_constraints(M_AT, aggregate=True, enforce_mixing_aggregate=True, enforce_mixing = False)

# Define objective function with define_objective
add_objective(M_AT)

In [178]:
M_AT.write('../models/MineProductionPlanningAT_real_pyomo.lp')    # model saved

# Choose an solver to optimize
opt = SolverFactory("gurobi", solver_io="python")

# Match critical parameters
opt.options.update({
    "NonConvex": 2,      # required for bilinear / general nonconvex quadratic constraints
    "MIPGap": 1e-4,      
    "OutputFlag": 1,   # print solver log
})
results = opt.solve(M_AT, tee=True)
print(results)
print("Objective value:", value(M_AT.Obj))

Set parameter OutputFlag to value 1
Set parameter NonConvex to value 2
Set parameter MIPGap to value 0.0001
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
NonConvex  2

Optimize a model with 7807 rows, 8755 columns and 56573 nonzeros
Model fingerprint: 0xec616d2c
Model has 1445 quadratic constraints
Variable types: 7310 continuous, 1445 integer (1445 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e-05, 3e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+07]
Found heuristic solution: objective -0.0000000
Presolve removed 1271 rows and 1396 columns
Presolve time: 0.21s
Presolved: 18333 rows, 9906 columns, 737

### (2) Aggregate Tracking Model (AT-IP)

$$
\begin{aligned}
S_{\mathrm{AT}\text{-}\mathrm{IP}}
:= \Big\{ & (x, y, z^{P}, z^{S}, z^{S,P}, z^{S,S}, o^{S}, a^{S}, o^{P}, a^{P}, f)
          \in [0,1]^{6(N\times T)} \times \mathbb{R}_{\ge 0}^{4T} \times [0,1]^T : \\
& \big([x], y, z^{P}, z^{S}, z^{S,P}, z^{S,S}, o^{S}, a^{S}, o^{P}, a^{P}, f\big)
   \text{ satisfies } (1)\text{--}(10),\ (12)\text{--}(17), \\
& x \in \{0,1\}^{N\times T} \Big\}.
\end{aligned}
$$

In [180]:
M_AT_IP = build_model(time_data, blocks_data, plants, stockpiles, waste_dumps, P, parameters)

# Define decision variables  with x is set to binary
add_variables(M_AT_IP, aggregate=True, x_binary=True)

# Define constraints with enforce_mixing=False
add_constraints(M_AT_IP, aggregate=True, enforce_mixing_aggregate=False, enforce_mixing = False)

# Define objective function with define_objective
add_objective(M_AT_IP)

In [182]:
M_AT_IP.write('../models/MineProductionPlanningAT_IP_real_pyomo.lp')    # model saved

# Choose an solver to optimize
opt = SolverFactory("gurobi", solver_io="python")

# Match critical parameters
opt.options.update({
    "NonConvex": 2,      # required for bilinear / general nonconvex quadratic constraints
    "MIPGap": 1e-4,      
    "OutputFlag": 1,   # print solver log
})
results = opt.solve(M_AT_IP, tee=True)
print(results)
print("Objective value:", value(M_AT_IP.Obj))

Set parameter OutputFlag to value 1
Set parameter NonConvex to value 2
Set parameter MIPGap to value 0.0001
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
NonConvex  2

Optimize a model with 7807 rows, 8755 columns and 56573 nonzeros
Model fingerprint: 0xe11cb89e
Variable types: 7310 continuous, 1445 integer (1445 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [1e-05, 3e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+07]
Found heuristic solution: objective -0.0000000
Presolve removed 1379 rows and 1629 columns
Presolve time: 0.32s
Presolved: 6428 rows, 7126 columns, 48344 nonzeros
Variable types: 6006 continuous, 1120 integer (1120 binary)

Root relaxation: objective 1.5656

### (3) Aggregate Tracking Model (AT-LP)

$$
\begin{aligned}
S_{\mathrm{AT}\text{-}\mathrm{LP}}
:= \Big\{ & (x, y, z^{P}, z^{S}, z^{S,P}, z^{S,S}, o^{S}, a^{S}, o^{P}, a^{P}, f)
          \in [0,1]^{6(N\times T)} \times \mathbb{R}_{\ge 0}^{4T} \times [0,1]^T : \\
& (x, y, z^{P}, z^{S}, z^{S,P}, z^{S,S}, o^{S}, a^{S}, o^{P}, a^{P}, f)
   \text{ satisfies } (1)\text{--}(10),\ (12)\text{--}(17) \Big\}.
\end{aligned}
$$

In [184]:
M_AT_LP = build_model(time_data, blocks_data, plants, stockpiles, waste_dumps, P, parameters)

# Define decision variables  with x is set to binary
add_variables(M_AT_LP, aggregate=True, x_binary=False)

# Define constraints with enforce_mixing=False
add_constraints(M_AT_LP, aggregate=True, enforce_mixing_aggregate=False, enforce_mixing = False)

# Define objective function with define_objective
add_objective(M_AT_LP)

In [186]:
M_AT_LP.write('../models/MineProductionPlanningAT_LP_real_pyomo.lp')    # model saved

# Choose an solver to optimize
opt = SolverFactory("gurobi", solver_io="python")

# Match critical parameters
opt.options.update({
    "NonConvex": 2,      # required for bilinear / general nonconvex quadratic constraints
    "MIPGap": 1e-4,      
    "OutputFlag": 1,   # print solver log
})
results = opt.solve(M_AT_LP, tee=True)
print(results)
print("Objective value:", value(M_AT_LP.Obj))

Set parameter OutputFlag to value 1
Set parameter NonConvex to value 2
Set parameter MIPGap to value 0.0001
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
NonConvex  2

Optimize a model with 7807 rows, 8755 columns and 56573 nonzeros
Model fingerprint: 0x34966cfa
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [1e-05, 3e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+07]
Presolve removed 1253 rows and 1451 columns
Presolve time: 0.04s
Presolved: 6554 rows, 7304 columns, 50087 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.04s

Barrier statistics:
 AA' NZ     : 1.469e+05
 Factor NZ  : 6.106e+05 (roughly 10 MB of memory)
 Fact